In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import joblib
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor

# Load the data
df = pd.read_csv('cleanedData.csv')

# Drop rows with missing target values
df = df.dropna(subset=['degerlendirme_puani'])

# Extract relevant columns for the classification model
cat_features = [
 'cinsiyet',
 'dogum_yeri',
 'ikametgah_sehri',
 'universite_adi',
 'universite_turu',
 'burslu_ise_burs_yuzdesi',
 'burs_aliyor_mu?',
 'universite_kacinci_sinif',
 'universite_not_ortalamasi',
 'daha_once_baska_bir_universiteden_mezun_olmus',
 'lise_adi',
 'lise_adi_diger',
 'lise_sehir',
 'lise_turu',
 'lise_mezuniyet_notu',
 'baska_bir_kurumdan_burs_aliyor_mu?',
 'burs_aldigi_baska_kurum',
 'baska_kurumdan_aldigi_burs_miktari',
 'anne_egitim_durumu',
 'anne_calisma_durumu',
 'baba_egitim_durumu',
 'baba_calisma_durumu',
 'kardes_sayisi',
 'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
 'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
 'aktif_olarak_bir_stk_uyesi_misiniz?',
 'stk_projesine_katildiniz_mi?',
 'girisimcilikle_ilgili_deneyiminiz_var_mi?',
 'ingilizce_biliyor_musunuz?',
 'ingilizce_seviyeniz?',
 'daha_onceden_mezun_olunduysa_mezun_olunan_universite',
 'anne_sektor_encoded',
 'baba_sektor_encoded',
 'anne_unknown',
 'anne_diger',
 'anne_kamu',
 'anne_ozel_sektor',
 'baba_unknown',
 'baba_diger',
 'baba_kamu',
 'baba_ozel_sektor',
 'age'
]

text_columns = [
    'girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz?', 
    'bolum', 
    'lise_bolumu', 
    'lise_bolum_diger',
    'uye_oldugunuz_kulubun_ismi',
    'spor_dalindaki_rolunuz_nedir?',
    "hangi_stk_nin_uyesisiniz?",
     ]
target_column = 'degerlendirme_puani'

# Handle missing values in text columns
df[text_columns] = df[text_columns].fillna('')

# Combine text columns into one for processing
df['combined_text'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Tokenize and preprocess text data for Word2Vec
def preprocess_text(text):
    return simple_preprocess(text, deacc=True)

# Check if the Word2Vec model already exists
import os
if os.path.exists('word2vec_model.model'):
    # Load the previously saved Word2Vec model
    word2vec_model = Word2Vec.load('word2vec_model.model')
else:
    # Prepare Word2Vec model
    sentences = [preprocess_text(text) for text in df['combined_text']]
    word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    # Save the Word2Vec model
    word2vec_model.save('word2vec_model.model')

# Create document vectors by averaging word vectors
def vectorize_text(text):
    tokens = preprocess_text(text)
    vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    if len(vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(vectors, axis=0)

df['text_vector'] = df['combined_text'].apply(vectorize_text)
word2vec_features = np.array(df['text_vector'].tolist())

# TF-IDF Vectorization
text_data = df['combined_text']
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_vectors = tfidf_vectorizer.fit_transform(text_data)

# Combine Word2Vec and TF-IDF features
combined_features = hstack([tfidf_vectors, word2vec_features])

# Debugging step: Verify columns in df and missing columns in cat_features
missing_cols = [col for col in cat_features if col not in df.columns]
if missing_cols:
    print("Warning: The following columns are missing from the DataFrame:", missing_cols)

# Prepare data for classification model
try:
    X_cat = df[cat_features]
except KeyError as e:
    print("Error: One or more columns are missing:", e)
    print("Available columns:", df.columns)
    raise

y = df[target_column]

# ColumnTransformer for encoding categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

# Classification pipeline
clf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Split data for classification model
X_cat_train, X_cat_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.2, random_state=42)

# Train classification model
clf_pipeline.fit(X_cat_train, y_train)

# Evaluate classification model
y_pred_cat = clf_pipeline.predict(X_cat_test)
print("Classification Model Accuracy:", accuracy_score(y_test, y_pred_cat))
print("Classification Report:\n", classification_report(y_test, y_pred_cat))

# Prepare data for regression model
# Debugging step: Print shapes and types
print("Shape of X_cat:", X_cat.shape)
print("Shape of combined_features:", combined_features.shape)
print("Type of X_cat:", type(X_cat))
print("Type of combined_features:", type(combined_features))
print("Dtype of X_cat:", X_cat.dtypes)
print("Dtype of combined_features:", combined_features.dtype)

# Identify numeric and categorical columns
numeric_features = X_cat.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_cat.select_dtypes(include=['object', 'bool']).columns

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_features)
    ])

# Fit and transform the data
X_cat_preprocessed = preprocessor.fit_transform(X_cat)

# Debugging step: Print shapes after preprocessing
print("Shape of X_cat_preprocessed:", X_cat_preprocessed.shape)
print("Shape of combined_features:", combined_features.shape)

# Combine features
X_combined = hstack([X_cat_preprocessed, combined_features])

# Debugging step: Print final combined shape
print("Shape of X_combined:", X_combined.shape)

# Split data for regression model
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Modify CatBoost hyperparameters
catboost_model = CatBoostRegressor(
    iterations=2000,  # Increased iterations
    learning_rate=0.05,  # Lowered learning rate
    depth=8,  # Increased depth
    l2_leaf_reg=3,  # Added L2 regularization
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=100  # Increased early stopping rounds
)

# Add feature selection
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(estimator=CatBoostRegressor(iterations=100), threshold='median')
X_selected = selector.fit_transform(X_combined, y)

# Split data using selected features
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the model with selected features
catboost_model.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    verbose=100
)

# Cross-validation
cv_scores = cross_val_score(catboost_model, X_combined, y, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validation RMSE: {(-cv_scores.mean())**0.5:.4f} (+/- {cv_scores.std() * 2:.4f})')

# Predict and evaluate regression model
y_pred_reg = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_reg)
rmse = mse ** 0.5
print(f'Regression Model RMSE: {rmse:.4f}')

# Print actual vs predicted values for regression
print("\nSample Actual vs. Predicted Values:")
for actual, predicted in zip(y_test[:10], y_pred_reg[:10]):
    print(f"Actual: {actual:.2f}, Predicted: {predicted:.2f}")

# Feature importance
feature_importance = catboost_model.feature_importances_
feature_names = preprocessor.get_feature_names_out().tolist() + [f'text_feature_{i}' for i in range(combined_features.shape[1])]
for name, importance in sorted(zip(feature_names, feature_importance), key=lambda x: x[1], reverse=True)[:20]:
    print(f"Feature: {name}, Importance: {importance:.4f}")

# Save the CatBoost model
catboost_model.save_model('catboost_model.cbm')

# Load the test data


Classification Model Accuracy: 0.19861080963750813
Classification Report:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         4
         2.0       0.00      0.00      0.00         2
         3.0       0.33      0.50      0.40         4
         4.0       0.33      0.27      0.30        11
         5.0       0.32      0.67      0.43        95
         6.0       0.35      0.76      0.48        86
         7.0       0.33      0.85      0.48       237
         8.0       0.47      0.56      0.51       154
         9.0       0.35      0.63      0.45       138
        10.0       0.29      0.25      0.27       134
        11.0       0.27      0.34      0.30       119
        12.0       0.37      0.42      0.40       274
        13.0       0.19      0.25      0.22       156
        14.0       0.34      0.31      0.32       236
        15.0       0.22      0.27      0.24       177
        16.0       0.36      0.33      0.35       188
      

C:\Users\Mustafa\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mustafa\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mustafa\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mustafa\anaconda3\Lib\site-packag

Shape of X_cat_preprocessed: (46068, 24842)
Shape of combined_features: (46068, 1100)
Shape of X_combined: (46068, 25942)
Learning rate set to 0.487525
0:	learn: 10.5797102	total: 371ms	remaining: 36.7s
1:	learn: 8.7626336	total: 502ms	remaining: 24.6s
2:	learn: 7.8845201	total: 638ms	remaining: 20.6s
3:	learn: 7.3810040	total: 768ms	remaining: 18.4s
4:	learn: 7.1276788	total: 898ms	remaining: 17.1s
5:	learn: 6.9477334	total: 1.02s	remaining: 16s
6:	learn: 6.8196844	total: 1.15s	remaining: 15.3s
7:	learn: 6.7026993	total: 1.28s	remaining: 14.7s
8:	learn: 6.6413156	total: 1.42s	remaining: 14.3s
9:	learn: 6.5953070	total: 1.54s	remaining: 13.9s
10:	learn: 6.5041347	total: 1.67s	remaining: 13.5s
11:	learn: 6.4599432	total: 1.81s	remaining: 13.2s
12:	learn: 6.4376301	total: 1.93s	remaining: 12.9s
13:	learn: 6.3593328	total: 2.06s	remaining: 12.7s
14:	learn: 6.3411291	total: 2.19s	remaining: 12.4s
15:	learn: 6.3283692	total: 2.31s	remaining: 12.2s
16:	learn: 6.3033431	total: 2.45s	remaining

In [ ]:
test_df = pd.read_csv('cleanTest.csv')

# Preprocess test data
test_df[text_columns] = test_df[text_columns].fillna('')
test_df['combined_text'] = test_df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Create document vectors for test data
test_df['text_vector'] = test_df['combined_text'].apply(vectorize_text)
test_word2vec_features = np.array(test_df['text_vector'].tolist())

# TF-IDF Vectorization for test data
test_tfidf_vectors = tfidf_vectorizer.transform(test_df['combined_text'])

# Combine Word2Vec and TF-IDF features for test data
test_combined_features = hstack([test_tfidf_vectors, test_word2vec_features])

# Prepare test data for the model
X_test_cat = test_df[cat_features]

# Preprocess test data
X_test_cat_preprocessed = preprocessor.transform(X_test_cat)

# Combine features for test data
X_test_combined = hstack([X_test_cat_preprocessed, test_combined_features])

# Apply feature selection
X_test_selected = selector.transform(X_test_combined)

# Load the saved CatBoost model
loaded_model = CatBoostRegressor()
loaded_model.load_model('catboost_model.cbm')

# Make predictions
test_predictions = loaded_model.predict(X_test_selected)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_df['id'],
    'Degerlendirme Puani': test_predictions
})

# Save the submission file
submission.to_csv('submission.csv', index=False)

print("Predictions have been saved to 'submission.csv'")
